# `bw_timex` Teaching Example - Time-explicit LCA of an Electric Vehicle

Here's a rundown of the steps involved in a `TimexLCA`:

<div style="display: flex; justify-content: center; background-color: white; border-radius: 15px; padding: 10px; width: 600px; margin: auto;">
  <img src="data/workflow.svg" style="border-radius: 15px; width: 100%;">
</div>

## **Step 0**: Initial Setup *(normal brightway stuff)*

In [ ]:
import bw2data as bd

bd.projects.set_current("bw25_ei310_premise")

In [ ]:
if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.register()

```{mermaid}
flowchart LR
    car_prod_wo_battery(market for passenger car, electric, without battery, GLO):::ei-->ev_production
    battery_production(market for battery, Li-ion, LiMn2O4, rechargeable, prismatic, GLO):::ei-->ev_production
    ev_production(ev production):::fg-->driving
    electricity_generation(market group for electricity, low voltage, GLO):::ei-->driving
    driving(driving ev):::fg-->battery_eol(market for used Li-ion battery, GLO):::ei
    driving-->fu(FU: driving 100,000 km)

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;
```

Legend:
```{mermaid}
flowchart TB
    background(background process):::ei
    foreground(foreground process):::fg

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;
```

### Creating foreground processes

In [ ]:
ev_production = foreground.new_node("ev_production", name="production of an electric vehicle", unit="unit")
ev_production['reference product'] = "electric vehicle"
ev_production.save()

driving = foreground.new_node("driving", name="driving an electric vehicle", unit="pkm over ev lifetime")
driving['reference product'] = "person transport"
driving.save()

### Adding exchanges

In [ ]:
ELECTRICITY_CONSUMPTION = 0.2 # kWh/km
MILEAGE = 100_000 # km
MASS_CAR_WITHOUT_BATTERY = 840 # kg
MASS_BATTERY = 280 # kg

In [ ]:
car_without_battery = bd.get_node(database="ei310_SSP2_RCP19_2025", name="market for passenger car, electric, without battery")
battery = bd.get_node(database="ei310_SSP2_RCP19_2025", name="market for battery, Li-ion, LiMn2O4, rechargeable, prismatic")
electricity = bd.get_node(database="ei310_SSP2_RCP19_2025", name="market group for electricity, low voltage", location="GLO")
battery_eol = bd.get_node(database="ei310_SSP2_RCP19_2025", name="market for used Li-ion battery")

In [ ]:
ev_production.new_edge(input=ev_production, amount=1, type="production").save()

car_without_battery_to_ev = ev_production.new_edge(
    input=car_without_battery,
    amount=MASS_CAR_WITHOUT_BATTERY,
    type="technosphere"
)
car_without_battery_to_ev.save()

battery_to_ev = ev_production.new_edge(
    input=battery, 
    amount=MASS_BATTERY, 
    type="technosphere"
)
battery_to_ev.save()

In [ ]:
driving.new_edge(input=driving, amount=1, type="production").save()

ev_to_driving = driving.new_edge(
    input=ev_production, 
    amount=1, 
    type="technosphere"
)
ev_to_driving.save()

electricity_to_driving = driving.new_edge(
    input=electricity,
    amount=ELECTRICITY_CONSUMPTION * MILEAGE,
    type="technosphere",
)
electricity_to_driving.save()

driving_to_battery_eol = driving.new_edge(
    input=battery_eol, 
    amount=-MASS_BATTERY, # used battery is "produced"
    type="technosphere"
)
driving_to_battery_eol.save()

With this setup, we could now run a normal, static LCA:

In [ ]:
import bw2calc as bc

method = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
lca = bc.LCA({driving: 1}, method)
lca.lci()
lca.lcia()
lca.score

Lateron, we'll want to use prospective background databases. We create those using premise. We save the info which database represent what point in time in a dictionary:

In [ ]:
from datetime import datetime

database_dates = {
    "ei310_SSP2_RCP19_2025": datetime.strptime("2020", "%Y"),
    "ei310_SSP2_RCP19_2030": datetime.strptime("2030", "%Y"),
    "ei310_SSP2_RCP19_2040": datetime.strptime("2040", "%Y"),
    "foreground": "dynamic", # Doesn't have a fixed date, but will be distributed over time
}

## **Step 1**: Adding temporal information

```{mermaid}
flowchart LR
    car_prod_wo_battery(market for passenger car, electric, without battery, GLO):::ei-->|1-2 years prior|ev_production
    battery_production(market for battery, Li-ion, LiMn2O4, rechargeable, prismatic, GLO):::ei-->|6-9 months prior|ev_production
    ev_production(ev production):::fg-->|1-3 months prior|driving
    electricity_generation(market group for electricity, low voltage, GLO):::ei-->|uniformly distributed over lifetime|driving
    driving(driving ev):::fg-->|3 months after lifetime|battery_eol(market for used Li-ion battery, GLO):::ei
    driving-->fu(**FU**: driving 100,000 km, *uniformly over lifetime, starting 2025*)

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;
```

### The `TemporalDistribution` class

In [ ]:
from bw_temporalis import TemporalDistribution
import numpy as np

td_car_without_battery_production = TemporalDistribution(
    date=np.array([-2, -1], dtype="timedelta64[Y]"), amount=np.array([0.2, 0.8])
)

td_assembly_and_delivery = TemporalDistribution(
    date=np.array([-3, -2, -1], dtype="timedelta64[M]"), amount=np.array([0.3, 0.5, 0.2])
)

#### What do these TDs look like?

In [ ]:
td_car_without_battery_production.graph(resolution="Y")

In [ ]:
td_assembly_and_delivery.graph(resolution="M")

In [ ]:
(td_car_without_battery_production * td_assembly_and_delivery).graph(resolution="M")

#### Creating the remaining TDs

In [ ]:
from bw_temporalis import easy_timedelta_distribution


td_battery_production = easy_timedelta_distribution(
    start=-9,
    end=-6,
    resolution="M",
    steps=4,
    kind="normal", # you can also use "uniform" or "triangular"
    param=0.5,
)

LIFETIME = 15 # years
td_electricity_consumption = easy_timedelta_distribution(
    start=0,
    end=LIFETIME,
    resolution="Y",
    steps=(LIFETIME + 1),
    kind="uniform",
)

td_battery_eol = TemporalDistribution(
    date=np.array([LIFETIME * 12 + 3], dtype="timedelta64[M]"), amount=np.array([1])
)


#### Adding TDs to the exchanges

In [ ]:
car_without_battery_to_ev['temporal_distribution'] = td_car_without_battery_production
car_without_battery_to_ev.save()

battery_to_ev['temporal_distribution'] = td_battery_production
battery_to_ev.save()

ev_to_driving['temporal_distribution'] = td_assembly_and_delivery
ev_to_driving.save()

# if you dont have the exchange object at hand, you can also use our util function:
from bw_timex.utils import add_temporal_distribution_to_exchange

add_temporal_distribution_to_exchange(
    td_electricity_consumption,
    input_node = electricity,
    output_node = driving,
)

add_temporal_distribution_to_exchange(
    td_battery_eol,
    input_node = battery_eol,
    output_node = driving,
)

## **Step 2**: Building the process timeline

In [ ]:
from bw_timex import TimexLCA

In [ ]:
tlca = TimexLCA({driving: 1}, method, database_dates)

In [ ]:
tlca.build_timeline(starting_datetime="2025-01-01", temporal_grouping="month")

## **Step 3**: Calculating the time-explicit LCI

In [ ]:
tlca.lci()

Under the hood, before solving the inventory problem, we're re-build the technosphere and biosphere matrices here. New rows and columns are added to carry the extra temporal information. Details are explained in the [Theory Section](https://docs.brightway.dev/projects/bw-timex/en/latest/content/theory.html#modifying-the-matrices) of our docs.

## **Step 4**: Impact assessment

The simplest case, static LCIA:

In [ ]:
tlca.static_lcia()
tlca.static_score

But there is more: `bw_timex` retains temporal information in the inventory, allowing for dynamic LCIA:

In [ ]:
tlca.dynamic_lcia(metric="GWP")

In [ ]:
from bw_timex.utils import plot_characterized_inventory_as_waterfall
plot_characterized_inventory_as_waterfall(tlca)

In addition to the "standard" GWP, we can go even more dynamic, directly assessing radiative forcing:

In [ ]:
tlca.dynamic_lcia(
    metric="radiative_forcing",
    time_horizon=30,
)

In [ ]:
tlca.plot_dynamic_characterized_inventory(
    sum_emissions_within_activity=True,
)